# Prevalence of PRIMIS codelists

In [1]:
import pandas as pd
import numpy as np
import os

suffix = f"_{os.environ.get('OPENSAFELY_BACKEND', 'tpp')}"
os.makedirs(os.path.join("..","safe-outputs","by-codelist"), exist_ok=True)

### Load data

In [2]:
dtypes={"sex":"category"}

df = pd.read_csv(os.path.join("..","output","input_with_codes.csv"), dtype=dtypes)

for col in df.columns:
    if col in ["patient_id", "age", "sex"]:
        continue
    if "_date" not in col: # float16 not suitable for snomed codes
        continue
    # Most columns only contain years or NaN so we can store them as
    # float16s, which saves a lot of memory
    df[col] = df[col].astype("float16")


### Create ageband

In [3]:
agebands = ['16-39', '40-69', '70+']
conditions = [
    (df['age'] >= 16) & (df['age'] < 40),
    (df['age'] >= 40) & (df['age'] < 70),
    (df['age'] >= 70) & (df['age'] < 120)]
choices = agebands
df['ageband'] = np.select(conditions, choices, default=np.nan)

# filter to largest sex groups
df['sex'] = np.where(df['sex'].isin(['I','U']), np.nan, df["sex"])

### Summarise data

In [4]:
# list columns of interest 
cols_allyears = [c for c in df.columns if ((c not in ["age","patient_id"]) & ("_date" not in c))]
cols_recent = ["preg", "pregdel"]

In [5]:
# filter to valid sexes and agegroups only
df1 = df.copy().loc[(df["sex"].isin(["F","M"])) & (df["ageband"].isin(agebands))]

### Calculate population denominators

In [6]:

out2 = df1.groupby(["ageband", "sex"])[["patient_id"]].nunique().rename(columns={"patient_id":"total_population"}).transpose()

# calculate total population across all ages and sexes
out2["total"] = out2.sum(axis=1)

out2

ageband          16-39       40-69        70+      total
sex                  F     M     F     M    F    M      
total_population  1590  1520  1863  1777  605  627  7982

### Codelist counts

In [7]:
# for codes that are only relevant if recent (pregnancy/delivery), remove any older dates
for c in cols_recent:
    df1.loc[(df1[f"{c}_date"]<2020), c] = np.nan

# exclude date columns
out = df1.copy()[cols_allyears]
    
# summarise most common codes for each codelist, by age and gender
for c in out.columns.drop(["sex", "ageband"]):
    # count number of occurrences for each code by ageband and sex:
    out = df1.groupby(["ageband", "sex",c])["patient_id"].nunique().reset_index()
    out = out.rename(columns={"patient_id":"patient_count"})
    
    out[c] = out[c].astype(int)
    
    # rank codes by number of occurrences
    out["rank"] = out.groupby(["ageband", "sex"])[["patient_count"]].rank(method="min", ascending=False)
    
    # keep top 10 codes and those with more than 10 occurrences only
    out = out[(out["rank"]<=10) & (out["patient_count"]>10)].drop("rank", axis=1)
    
    
    # join population denominators
    out = out.set_index(["ageband", "sex"])
    out = out.join(out2.transpose())    

    # calculate rates
    out["rate_per_1000"] = (1000*(out["patient_count"]/out["total_population"]))
    
    # round based on values
    dp = 1
    if out["rate_per_1000"].max() <1:
        dp = 2
    out["rate_per_1000"] = out["rate_per_1000"].round(dp)
    
    out = out.drop(["patient_count","total_population"], 1)

    display(out)

    # export to csv    
    out.to_csv(os.path.join("..","safe-outputs","by-codelist",f"code-prevalence-by-age-and-sex_{c}_{suffix}.csv"))



registered  rate_per_1000
ageband sex                           
16-39   F             0           47.8
        F             1          952.2
        M             0           50.0
        M             1          950.0
40-69   F             0           50.5
        F             1          949.5
        M             0           48.4
        M             1          951.6
70+     F             0           54.5
        F             1          945.5
        M             0           52.6
        M             1          947.4

bmi_stage  rate_per_1000
ageband sex                          
16-39   F     35425004           43.4
        F    819948005           52.2
        M     35425004           45.4
        M    819948005           41.4
40-69   F     35425004           47.8
        F    819948005           47.2
        M     35425004           56.8
        M    819948005           52.3
70+     F     35425004           49.6
        F    819948005           52.9
        M     35425004           44.7
        M    819948005           54.2

carehome  rate_per_1000
ageband sex                                 
16-39   F    1092561000000107           93.7
        M    1092561000000107          100.0
40-69   F    1092561000000107          102.0
        M           158944006            6.2
        M    1092561000000107           93.4
70+     F    1092561000000107           79.3
        M    1092561000000107           89.3

carer  rate_per_1000
ageband sex                          
16-39   F    276048003           55.3
        F    407542009           49.1
        M    276048003           43.4
        M    407542009           53.9
40-69   F    276048003           41.9
        F    407542009           42.4
        M    276048003           53.5
        M    407542009           59.1
70+     F    276048003           56.2
        F    407542009           52.9
        M    276048003           44.7
        M    407542009           49.4

notcarer  rate_per_1000
ageband sex                                
16-39   F    199361000000101           50.9
        F    506401000000109           49.1
        M    199361000000101           51.3
        M    506401000000109           38.8
40-69   F    199361000000101           54.2
        F    506401000000109           54.2
        M    199361000000101           49.5
        M    506401000000109           54.0
70+     F    199361000000101           44.6
        F    506401000000109           51.2
        M    199361000000101           52.6
        M    506401000000109           57.4

,,preg,rate_per_1000
ageband,sex,,


,,pregdel,rate_per_1000
ageband,sex,,


sev_mental  rate_per_1000
ageband sex                           
16-39   F      13746004           42.1
        F      49468007           52.8
        M      13746004           67.8
        M      49468007           43.4
40-69   F      13746004           47.2
        F      49468007           53.1
        M      13746004           46.7
        M      49468007           46.7
70+     F      13746004           69.4
        F      49468007           33.1
        M      13746004           55.8
        M      49468007           47.8

sev_obesity  rate_per_1000
ageband sex                                
16-39   F          408512008           28.3
        F          819948005           31.4
        F    914741000000103           28.3
        M          408512008           28.3
        M          819948005           33.6
        M    914741000000103           37.5
40-69   F          408512008           36.0
        F          819948005           29.5
        F    914741000000103           30.6
        M          408512008           32.1
        M          819948005           37.1
        M    914741000000103           33.2
70+     F          408512008           41.3
        F          819948005           41.3
        F    914741000000103           24.8
        M          408512008           28.7
        M          819948005           39.9
        M    914741000000103           47.8

smhres  rate_per_1000
ageband sex                          
16-39   F     41836007           25.2
        F    698951002           22.0
        M     41836007           27.0
        M    698951002           21.7
40-69   F     41836007           26.3
        F    698951002           23.1
        M     41836007           21.9
        M    698951002           27.6
70+     F     41836007           23.1
        F    698951002           21.5
        M     41836007           33.5
        M    698951002           33.5